In [19]:
import os
import pandas as pd
import yfinance as yf
from datetime import datetime
import sys
sys.path.append(os.getcwd() + "/../src")
from config import load_env, get_key

load_env()
print("DATA_DIR =", get_key("DATA_DIR"))

DATA_DIR = .\data


In [20]:
ticker = "JPM"  # JPMorgan
df = yf.download(ticker, period="6mo", interval="1d")

df.reset_index(inplace=True)
df["Date"] = pd.to_datetime(df["Date"])

# Validation
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("NA counts:\n", df.isna().sum())

# Save
os.makedirs("../data/raw", exist_ok=True)
fname = f"../data/raw/api_yfinance_{ticker}_{datetime.now().strftime('%Y%m%d-%H%M')}.csv"
df.to_csv(fname, index=False)
fname


C:\Users\ESHA\AppData\Local\Temp\ipykernel_26880\722505798.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="6mo", interval="1d")
[*********************100%***********************]  1 of 1 completed

Shape: (125, 6)
Columns: [('Date', ''), ('Close', 'JPM'), ('High', 'JPM'), ('Low', 'JPM'), ('Open', 'JPM'), ('Volume', 'JPM')]
NA counts:
 Price   Ticker
Date              0
Close   JPM       0
High    JPM       0
Low     JPM       0
Open    JPM       0
Volume  JPM       0
dtype: int64


'../data/raw/api_yfinance_JPM_20250821-0353.csv'

In [25]:
!pip3 install lxml


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import requests
import lxml
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html.parser")

table = soup.find("table", {"id": "constituents"})
rows = table.find_all("tr")

data_rows = []
for row in rows[1:]:
    cols = [c.get_text(strip=True) for c in row.find_all("td")]
    if len(cols) >= 5:
        data_rows.append([cols[0], cols[1], cols[3], cols[4], cols[5]])

df_sp500 = pd.DataFrame(data_rows, columns=["symbol","security","sector","sub_industry","hq"])

# Validation
print(df_sp500.info())
print("NA counts:\n", df_sp500.isna().sum())

# Save
fname_sp = f"../data/raw/scrape_wikipedia_sp500_{datetime.now().strftime('%Y%m%d-%H%M')}.csv"
df_sp500.to_csv(fname_sp, index=False)
fname_sp


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   symbol        503 non-null    object
 1   security      503 non-null    object
 2   sector        503 non-null    object
 3   sub_industry  503 non-null    object
 4   hq            503 non-null    object
dtypes: object(5)
memory usage: 19.8+ KB
None
NA counts:
 symbol          0
security        0
sector          0
sub_industry    0
hq              0
dtype: int64


'../data/raw/scrape_wikipedia_sp500_20250821-0358.csv'

### Data Sources
- API: Yahoo Finance (`yfinance`), ticker = JPM (banking exposure proxy).
- Scrape: Wikipedia S&P 500 constituents table.

### Validation
- API: Checked required columns (`Date`, `Close`), NA counts.
- Scrape: Checked text/numeric columns, NA counts.

### Assumptions & Risks
- `yfinance` schema may change or rate-limit.
- Wikipedia table format may shift (scraping fragile).
- `.env` not committed; `.env.example` included for reproducibility.